# Notebook 01 — Load, Clean, and Prepare Data (May 2024)

## Objective
Prepare an analysis-ready dataset for Cyclistic (Divvy) May 2024 trip data by:
- validating the raw structure,
- cleaning obvious issues,
- creating engineered features needed for analysis,
- saving a cleaned dataset for subsequent notebooks.

## Business context (1 sentence)
We are preparing data to compare how **casual riders** and **annual members** use the bike-share service differently, to support marketing strategies aimed at converting casual riders into members.



In [2]:
import pandas, numpy, matplotlib, seaborn
print("Environment OK")


Environment OK


In [3]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 140)



In [4]:
raw_path = "../data_raw/202405-divvy-tripdata.csv"
df = pd.read_csv(raw_path)

df.head()



,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,7D9F0CE9EC2A1297,classic_bike,2024-05-25 15:52:42,2024-05-25 16:11:50,Streeter Dr & Grand Ave,13022,Clark St & Elm St,TA1307000039,41.892278,-87.612043,41.902973,-87.631280,casual
1,02EC47687411416F,classic_bike,2024-05-14 15:11:51,2024-05-14 15:22:00,Sheridan Rd & Greenleaf Ave,KA1504000159,Sheridan Rd & Loyola Ave,RP-009,42.010587,-87.662412,42.001044,-87.661198,casual
2,101370FB2D3402BE,classic_bike,2024-05-30 17:46:04,2024-05-30 18:09:16,Streeter Dr & Grand Ave,13022,Wabash Ave & 9th St,TA1309000010,41.892278,-87.612043,41.870769,-87.625734,member
3,E97E396331ED6913,electric_bike,2024-05-17 20:21:54,2024-05-17 20:40:32,Streeter Dr & Grand Ave,13022,Sheffield Ave & Wellington Ave,TA1307000052,41.892270,-87.611946,41.936253,-87.652662,member
4,674EDE311C543165,classic_bike,2024-05-22 18:52:20,2024-05-22 18:59:04,Larrabee St & Division St,KA1504000079,Clark St & Elm St,TA1307000039,41.903486,-87.643353,41.902973,-87.631280,casual


In [5]:
print("Shape:", df.shape)
df.info()



Shape: (609493, 13)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 609493 entries, 0 to 609492
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ride_id             609493 non-null  object 
 1   rideable_type       609493 non-null  object 
 2   started_at          609493 non-null  object 
 3   ended_at            609493 non-null  object 
 4   start_station_name  500445 non-null  object 
 5   start_station_id    500445 non-null  object 
 6   end_station_name    496762 non-null  object 
 7   end_station_id      496762 non-null  object 
 8   start_lat           609493 non-null  float64
 9   start_lng           609493 non-null  float64
 10  end_lat             608709 non-null  float64
 11  end_lng             608709 non-null  float64
 12  member_casual       609493 non-null  object 
dtypes: float64(4), object(9)
memory usage: 60.5+ MB


In [6]:
(df.isna().mean().sort_values(ascending=False) * 100).round(2)



end_station_name      18.50
end_station_id        18.50
start_station_id      17.89
start_station_name    17.89
end_lat                0.13
end_lng                0.13
started_at             0.00
rideable_type          0.00
ride_id                0.00
ended_at               0.00
start_lat              0.00
start_lng              0.00
member_casual          0.00
dtype: float64

In [7]:
df["started_at"] = pd.to_datetime(df["started_at"], errors="coerce")
df["ended_at"] = pd.to_datetime(df["ended_at"], errors="coerce")

df[["started_at", "ended_at"]].isna().sum()



started_at    0
ended_at      0
dtype: int64

In [8]:
df["ride_length_minutes"] = (df["ended_at"] - df["started_at"]).dt.total_seconds() / 60

df["ride_length_minutes"].describe()


count    609493.000000
mean         18.883762
std          63.455143
min       -2748.316667
25%           5.883333
50%          10.533333
75%          19.066667
max        1499.950000
Name: ride_length_minutes, dtype: float64

In [9]:
before = df.shape[0]

# Remove rows where datetime conversion failed or duration <= 0
df = df.dropna(subset=["started_at", "ended_at", "ride_length_minutes"])
df = df[df["ride_length_minutes"] > 0]

after = df.shape[0]
print(f"Removed {before - after:,} invalid rides. Remaining: {after:,}")
df["ride_length_minutes"].describe()


Removed 261 invalid rides. Remaining: 609,232


count    609232.000000
mean         18.896934
std          63.368004
min           0.016667
25%           5.900000
50%          10.533333
75%          19.066667
max        1499.950000
Name: ride_length_minutes, dtype: float64

In [10]:
df["day_of_week"] = df["started_at"].dt.day_name()
df["hour"] = df["started_at"].dt.hour
df["date"] = df["started_at"].dt.date
df["is_weekend"] = df["day_of_week"].isin(["Saturday", "Sunday"])

df[["started_at", "day_of_week", "hour", "is_weekend", "ride_length_minutes"]].head()


,started_at,day_of_week,hour,is_weekend,ride_length_minutes
0,2024-05-25 15:52:42,Saturday,15,True,19.133333
1,2024-05-14 15:11:51,Tuesday,15,False,10.150000
2,2024-05-30 17:46:04,Thursday,17,False,23.200000
3,2024-05-17 20:21:54,Friday,20,False,18.633333
4,2024-05-22 18:52:20,Wednesday,18,False,6.733333


## Cleaning and Feature Engineering Log

Steps applied:
1. Loaded May 2024 trip data (each row = one ride).
2. Converted `started_at` and `ended_at` to datetime.
3. Created `ride_length_minutes` from end-start timestamps.
4. Removed invalid rides where:
   - timestamps failed to parse, or
   - ride length was zero/negative.
5. Created analysis features:
   - `day_of_week` (weekday name),
   - `hour` (start hour),
   - `date` (start date),
   - `is_weekend` (Saturday/Sunday flag).

Notes:
- Station fields have missing values; this is expected in Divvy trip data and does not prevent comparing members vs casual riders.


In [11]:
print("Final shape:", df.shape)
print("Columns:", df.columns.tolist())

output_path = "../data_cleaned/cyclistic_may2024_cleaned.csv"
df.to_csv(output_path, index=False)

print(f"Cleaned dataset saved to: {output_path}")


Final shape: (609232, 18)
Columns: ['ride_id', 'rideable_type', 'started_at', 'ended_at', 'start_station_name', 'start_station_id', 'end_station_name', 'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng', 'member_casual', 'ride_length_minutes', 'day_of_week', 'hour', 'date', 'is_weekend']
Cleaned dataset saved to: ../data_cleaned/cyclistic_may2024_cleaned.csv
